In [1]:
import endomill
from nbmetalog import nbmetalog as nbm


In [2]:
nbm.print_metadata()


context: ci
hostname: 757f2e110f56
interpreter: 3.8.10 (default, Nov 22 2023, 10:22:35)  [GCC 9.4.0]
nbcellexec: 2
nbname: phylogeny_simulation_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_simulation_mill.ipynb
revision: null
session: 7e04c7d5-36c6-4514-bcad-52ef50a8fbd9
timestamp: 2024-03-17T00:39:27Z00:00


IPython==7.16.1
keyname==0.4.1
yaml==5.3.1
endomill==0.1.3
nbmetalog==0.2.6
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.3.0
json==2.0.9
ipykernel==5.5.3


In [3]:
endomill.instantiate_over(
    parameter_packs=[
        {
            'data_url': data_url,
            'data_filename': data_filename,
        }
        for data_filename, data_url in [
            (
                'nk_ecoeaselection_seed110_pop100_mut.01_snapshot_3000.csv',
                'https://osf.io/5d3be/',
            ),
            (
                'nk_lexicaseselection_seed110_pop165_mut.01_snapshot_500.csv',
                'https://osf.io/8ycq7/',
            ),
            (
                'nk_randomselection_seed7_pop100_mut.01_snapshot_5000.csv',
                'https://osf.io/ydxt7/',
            ),
            (
                'nk_sharingselection_seed10_pop100_mut.01_snapshot_5000.csv',
                'https://osf.io/cz9fk/',
            ),
            (
                'nk_tournamentselection_seed140_pop100_mut.01_snapshot_5000.csv',
                'https://osf.io/5ubn8/',
            )
        ]
    ],
)


detected executing.endomill.ipynb file
skipping instantiate_over


In [4]:
# define papermill parameters
data_url: str
data_filename: str


In [5]:
# Parameters
data_url = "https://osf.io/cz9fk/"
data_filename = "nk_sharingselection_seed10_pop100_mut.01_snapshot_5000.csv"


In [6]:
import anytree
from bs4 import BeautifulSoup
from collections import defaultdict
from hstrat import hstrat
from interval_search import doubling_search
from iterpop import iterpop as ip
import itertools as it
from keyname import keyname as kn
import numpy as np
import opytional as opyt
import pandas as pd
import random
import sys
from tqdm import tqdm
from urllib import request


In [7]:
random.seed(1) # ensure reproducibility
sys.setrecursionlimit(100000) # anytree exceeds default recursion limit on our data


In [8]:
from nbmetalog import nbmetalog as nbm
nbm.print_metadata()


context: ci
hostname: 757f2e110f56
interpreter: 3.8.10 (default, Nov 22 2023, 10:22:35)  [GCC 9.4.0]
nbcellexec: 8
nbname: phylogeny_simulation_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_simulation_mill.ipynb
revision: null
session: 7e04c7d5-36c6-4514-bcad-52ef50a8fbd9
timestamp: 2024-03-17T00:39:27Z00:00


IPython==7.16.1
endomill==0.1.3
ipykernel==5.5.3
ipython_genutils==0.2.0
json==2.0.9
keyname==0.4.1
logging==0.5.1.2
nbmetalog==0.2.6
re==2.2.1
yaml==5.3.1
zmq==22.3.0
anytree==2.8.0
hstrat==0.3.2
iterpop==0.4.0
numpy==1.21.5
opytional==0.1.0
pandas==2.0.3


In [9]:
endomill.add_instance_outpath(
    f'a=phylogeny_simulation+source={data_filename}.endomill.ipynb',
)


# Retrieve Target Phylogeny from OSF


In [10]:
target_phylogeny_df = pd.read_csv(
    f'{data_url}/download',
)

nbm.print_dataframe_synopsis(target_phylogeny_df)


digest: eb5c427177485bfdabc5055e6a2eff557af995416bcd8a6165cf3c9fb86b3f04
manifest:
  ancestor_list: '    3241#  ex., [316428]'
  depth: '            3048#  ex., 3042'
  destruction_time: ' 2547#  ex., inf'
  id: '               3313#  ex., 316653'
  num_offspring: '    6#     ex., 0'
  num_orgs: '         3#     ex., 1'
  origin_time: '      3057#  ex., 5000'
  phenotype: '        3297#  ex., [ 0.600023 0.795695 0.965585 0.391101 0.863194 0.982911
    0.43849 0.470596 0.690861 0.73147 0.936691 0.993888 0.891655 0.889409 0.120467
    0.918155 0.649416 0.985481 0.545775 0.298241 0.0437687 0.00350497 0.882206 0.411082
    0.641201 0.171095 0.713238 0.633266 0.28839 0.159282 0.854242 0.975637 0.947336
    0.949524 0.970265 0.26913 0.342512 0.973407 0.68399 0.507257 0.897186 0.263287
    0.611318 0.628408 0.961238 0.395228 0.281177 0.992705 0.577607 0.00370154 0.582243
    0.242543 0.0358797 0.847181 0.0894643 0.492351 0.713603 0.176921 0.459707 0.661723
    0.478606 0.402292 0.69406 0.4372

# Create a Tree with Target Phylogeny Structure


In [11]:
# map id to anytree node
nodes = defaultdict(anytree.AnyNode)
for __, row in target_phylogeny_df.iterrows():
    node = nodes[row['id']]
    node.id = row['id']
    node.origin_time = row['origin_time']
    if 'NONE' not in row['ancestor_list']:
        ancestor_id = ip.popsingleton(
            eval(row['ancestor_list'])
        )
        node.parent = nodes[ancestor_id]

roots = {node.root for node in nodes.values()}


In [12]:
root = ip.popsingleton(roots)
root.height


3047

In [13]:
max_origin_time = target_phylogeny_df['origin_time'].max()
max_origin_time


5000

In [14]:
mean_leaf_origin_time = np.mean([
    leaf.origin_time
    for leaf in root.leaves
])
mean_leaf_origin_time


4999.890410958904

# Pick Parameters for Hereditary Stratigraphic Columns


In [15]:
def make_conditions(num_generations: int) -> pd.DataFrame:
    res = []
    for condemner_factory, target_column_bits, differentia_bit_width in it.product(
        [
            hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution,
            hstrat.StratumRetentionCondemnerRecencyProportionalResolution,
        ],
        [
            64,
            64 * 8,
            64 * 64,
        ],
        [
            1,
            8,
            64,
        ],
    ):
        policy_param = doubling_search(
            lambda x: \
                condemner_factory(x + 1).CalcNumStrataRetainedExact(num_generations)
                * differentia_bit_width > target_column_bits or x >= num_generations,
            {
                hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution: 1,
                hstrat.StratumRetentionCondemnerRecencyProportionalResolution: 0,
            }[condemner_factory],
        )

        actual_column_strata = condemner_factory(policy_param).CalcNumStrataRetainedExact(num_generations)
        actual_column_bits = actual_column_strata * differentia_bit_width

        res.append({
            'Retention Policy' : condemner_factory.__name__[25:],
            'Differentia Bit Width' : differentia_bit_width,
            'Retention Policy Resolution Parameter' : policy_param,
            'Target Retained Bits' : target_column_bits,
            'Actual Retained Bits' : actual_column_bits,
            'Retained Bits Error' : actual_column_bits - target_column_bits,
            'Actual Retained Strata' : actual_column_strata,
            'condemner' : condemner_factory(policy_param),
        })
    return pd.DataFrame.from_records(res)


In [16]:
conditions_df = make_conditions(int(mean_leaf_origin_time))
conditions_df.drop('condemner', axis=1)


,Retention Policy,Differentia Bit Width,Retention Policy Resolution Parameter,Target Retained Bits,Actual Retained Bits,Retained Bits Error,Actual Retained Strata
0,TaperedDepthProportionalResolution,1,31,64,63,-1,63
1,TaperedDepthProportionalResolution,8,3,64,56,-8,7
2,TaperedDepthProportionalResolution,64,1,64,192,128,3
3,TaperedDepthProportionalResolution,1,255,512,511,-1,511
4,TaperedDepthProportionalResolution,8,31,512,504,-8,63
5,TaperedDepthProportionalResolution,64,3,512,448,-64,7
6,TaperedDepthProportionalResolution,1,2048,4096,4096,0,4096
7,TaperedDepthProportionalResolution,8,255,4096,4088,-8,511
8,TaperedDepthProportionalResolution,64,31,4096,4032,-64,63
9,RecencyProportionalResolution,1,5,64,59,-5,59


# Set Up Ancestor Column


In [17]:
bundle = hstrat.HereditaryStratigraphicColumnBundle({
    kn.pack({
        'differentia' : row['Differentia Bit Width'],
        'policy' : row['Retention Policy'],
        'resolution' : row['Retention Policy Resolution Parameter'],
        'target_bits' : row['Target Retained Bits'],
        'actual_bits' : row['Actual Retained Bits'],
        'bits_error' : row['Retained Bits Error'],
        'actual_strata' : row['Actual Retained Strata'],
    }) \
        : hstrat.HereditaryStratigraphicColumn(
            stratum_differentia_bit_width=row['Differentia Bit Width'],
            stratum_retention_condemner=row['condemner'],
    )
    for __, row in conditions_df.iterrows()
})


# Simulate Inheritance of Ancestor Column Down Phylogenetic Tree


In [18]:
root = ip.popsingleton(roots)
root.hstrat_column = bundle

for node in anytree.LevelOrderIter(ip.popsingleton(roots)):
    parent = node.parent
    if parent is not None:
        node.hstrat_column = parent.hstrat_column.Clone()
        for __ in range(node.origin_time - parent.origin_time): node.hstrat_column.DepositStratum()


# Extract Pairwise MRCA Estimates for Extant Organisms


In [19]:
# impl -> mean retained bits
mean_retained_bits = {
    impl: np.mean([
        node.hstrat_column.GetNumStrataRetained()[impl]
        * int(kn.unpack(impl)['differentia'])
        for node in root.leaves
    ])
    for impl in bundle
}

records = [
    {
        **{
            'Mean Actual Retained Bits': v,
        },
        **kn.unpack(k)
    }
    for k, v in mean_retained_bits.items()
]
actual_retained_bits_df = pd.DataFrame.from_records(records)
actual_retained_bits_df.to_csv(
    f'a=actual_retained_bits+source={data_filename}',
)

actual_retained_bits_df


,Mean Actual Retained Bits,actual_bits,actual_strata,bits_error,differentia,policy,resolution,target_bits,_
0,63.000000,63,63,-1,1,TaperedDepthProportionalResolution,31,64,actual_bits=63+actual_strata=63+bits_error=-1+...
1,56.000000,56,7,-8,8,TaperedDepthProportionalResolution,3,64,actual_bits=56+actual_strata=7+bits_error=-8+d...
2,192.000000,192,3,128,64,TaperedDepthProportionalResolution,1,64,actual_bits=192+actual_strata=3+bits_error=128...
3,510.095890,511,511,-1,1,TaperedDepthProportionalResolution,255,512,actual_bits=511+actual_strata=511+bits_error=-...
4,504.000000,504,63,-8,8,TaperedDepthProportionalResolution,31,512,actual_bits=504+actual_strata=63+bits_error=-8...
5,448.000000,448,7,-64,64,TaperedDepthProportionalResolution,3,512,actual_bits=448+actual_strata=7+bits_error=-64...
6,4096.000000,4096,4096,0,1,TaperedDepthProportionalResolution,2048,4096,actual_bits=4096+actual_strata=4096+bits_error...
7,4080.767123,4088,511,-8,8,TaperedDepthProportionalResolution,255,4096,actual_bits=4088+actual_strata=511+bits_error=...
8,4032.000000,4032,63,-64,64,TaperedDepthProportionalResolution,31,4096,actual_bits=4032+actual_strata=63+bits_error=-...
9,58.178082,59,59,-5,1,RecencyProportionalResolution,5,64,actual_bits=59+actual_strata=59+bits_error=-5+...


In [20]:
res = []
for extant1, extant2 in tqdm([*it.product(root.leaves, root.leaves)]):
    if extant1 != extant2:
        bounds = extant1.hstrat_column.CalcRankOfMrcaBoundsWith(extant2.hstrat_column)
        for impl in extant1.hstrat_column:
            res.append({
                'Column Configuration' \
                    : impl,
                'Differentia Bit Width' \
                    : kn.unpack(impl)['differentia'],
                'Stratum Retention Policy' \
                    : kn.unpack(impl)['policy'],
                'Stratum Retention Policy Resolution Parameter' \
                    : kn.unpack(impl)['resolution'],
                'Stratigraphic Column Expected Retained Bits' \
                    : kn.unpack(impl)['actual_bits'],
                'Stratigraphic Column Mean Actual Retained Bits' \
                    : mean_retained_bits[impl],
                'Stratigraphic Column Target Retained Bits' \
                    : kn.unpack(impl)['target_bits'],
                'Stratigraphic Column Expected Retained Bits Error' \
                    : kn.unpack(impl)['bits_error'],
                'Stratigraphic Column Actual Num Retained Strata' \
                    : kn.unpack(impl)['actual_strata'],
                'Taxon Compared From' \
                    : extant1.id,
                'Taxon Compared To' \
                    : extant2.id,
                'Generation of Taxon Compared From' \
                    : extant1.hstrat_column.GetNumStrataDeposited(),
                'Generation of Taxon Compared To' \
                    : extant2.hstrat_column.GetNumStrataDeposited(),
                'Generation Of MRCA Lower Bound (inclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[0],
                    ),
                'Generation Of MRCA Upper Bound (exclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[1],
                    ),
                'MRCA Bound Confidence' \
                    : extant1.hstrat_column[impl].CalcRankOfMrcaBoundsWithProvidedConfidenceLevel(),
                'Rank of Earliest Detectable Mrca With' \
                    : extant1.hstrat_column[impl].CalcRankOfEarliestDetectableMrcaWith(extant2.hstrat_column[impl]),
            })

res_df = pd.DataFrame.from_records(res)


  0%|          | 0/5329 [00:00<?, ?it/s]

  0%|          | 3/5329 [00:00<03:20, 26.60it/s]

  0%|          | 5/5329 [00:00<03:49, 23.15it/s]

  0%|          | 7/5329 [00:00<04:08, 21.38it/s]

  0%|          | 9/5329 [00:00<04:23, 20.19it/s]

...

100%|█████████▉| 5321/5329 [04:43<00:00, 19.55it/s]

100%|█████████▉| 5323/5329 [04:43<00:00, 19.39it/s]

100%|█████████▉| 5325/5329 [04:43<00:00, 19.43it/s]

100%|█████████▉| 5327/5329 [04:43<00:00, 19.38it/s]

100%|██████████| 5329/5329 [04:43<00:00, 18.78it/s]

In [21]:
res_df


,Column Configuration,Differentia Bit Width,Stratum Retention Policy,Stratum Retention Policy Resolution Parameter,Stratigraphic Column Expected Retained Bits,Stratigraphic Column Mean Actual Retained Bits,Stratigraphic Column Target Retained Bits,Stratigraphic Column Expected Retained Bits Error,Stratigraphic Column Actual Num Retained Strata,Taxon Compared From,Taxon Compared To,Generation of Taxon Compared From,Generation of Taxon Compared To,Generation Of MRCA Lower Bound (inclusive),Generation Of MRCA Upper Bound (exclusive),MRCA Bound Confidence,Rank of Earliest Detectable Mrca With
0,actual_bits=63+actual_strata=63+bits_error=-1+...,1,TaperedDepthProportionalResolution,31,63,63.000000,64,-1,63,316622,316618,5001,5001,4800,5001,0.968750,512
1,actual_bits=56+actual_strata=7+bits_error=-8+d...,8,TaperedDepthProportionalResolution,3,56,56.000000,64,-8,7,316622,316618,5001,5001,4608,5000,0.996094,0
2,actual_bits=192+actual_strata=3+bits_error=128...,64,TaperedDepthProportionalResolution,1,192,192.000000,64,128,3,316622,316618,5001,5001,4096,5000,1.000000,0
3,actual_bits=511+actual_strata=511+bits_error=-...,1,TaperedDepthProportionalResolution,255,511,510.095890,512,-1,511,316622,316618,5001,5001,4960,5000,0.968750,64
4,actual_bits=504+actual_strata=63+bits_error=-8...,8,TaperedDepthProportionalResolution,31,504,504.000000,512,-8,63,316622,316618,5001,5001,4992,5000,0.996094,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94603,actual_bits=472+actual_strata=59+bits_error=-4...,8,RecencyProportionalResolution,5,472,465.424658,512,-40,59,316643,316640,5001,5001,4999,5000,0.996094,0
94604,actual_bits=448+actual_strata=7+bits_error=-64...,64,RecencyProportionalResolution,0,448,395.397260,512,-64,7,316643,316640,5001,5001,4992,5000,1.000000,0
94605,actual_bits=4096+actual_strata=4096+bits_error...,1,RecencyProportionalResolution,1596,4096,4096.986301,4096,0,4096,316643,316640,5001,5001,4995,5000,0.968750,8
94606,actual_bits=4056+actual_strata=507+bits_error=...,8,RecencyProportionalResolution,71,4056,4049.424658,4096,-40,507,316643,316640,5001,5001,4999,5000,0.996094,0


In [22]:
nbm.print_dataframe_synopsis(res_df)


digest: df1610ca5746491e194f5807f3b6f82e35c7bf2618c4cf48e69db3d1b08f32d7
manifest:
  Column Configuration: '                              18#  ex., actual_bits=63+actual_strata=63+bits_error=-1+differentia=1+policy=TaperedDepthProportionalResolution+resolution=31+target_bits=64'
  Differentia Bit Width: '                             3#   ex., 1'
  Generation Of MRCA Lower Bound (inclusive): '        45#  ex., 4800'
  Generation Of MRCA Upper Bound (exclusive): '        32#  ex., 5001'
  Generation of Taxon Compared From: '                 3#   ex., 5001'
  Generation of Taxon Compared To: '                   3#   ex., 5001'
  MRCA Bound Confidence: '                             3#   ex., 0.96875'
  Rank of Earliest Detectable Mrca With: '             6#   ex., 512'
  Stratigraphic Column Actual Num Retained Strata: '   7#   ex., 63'
  Stratigraphic Column Expected Retained Bits: '       14#  ex., 63'
  Stratigraphic Column Expected Retained Bits Error: ' 9#   ex., -1'
  Stratigraphic C

# Save Pairwise MRCA Estimates to File


In [23]:
res_df.to_csv(
    f'a=pairwise_mrca_estimates+source={data_filename}.gz',
    compression='gzip',
)
res_df.to_csv(
    f'a=pairwise_mrca_estimates+source={data_filename}',
)
